## Agenda

+ Introduction and setup environment (credentials)
+ Platform Setup and Walkthrough
+ Explanation of and Examination of Thunderbird Ground Truth
+ Model Selection ( 1 encoder/decoder), (2 decoder), eval against GPT4
+ Run experiment and show results
+ Evaluate results and discuss

## Foxfooding Introduction and Setup Environment

## Who we are, what we do, about the platform, etc. 


## Platform Setup and Walkthrough

In [1]:
# TODO Replace with ELB URL 
API_URL = "http://127.0.0.1/api/v1"

In [36]:
import requests
from typing import Dict, Any
import json

def make_request(
    url: str,
    method: str = "GET",
    params: Dict[str, Any] = None,
    data: Dict[str, Any] = None,
    headers: Dict[str, str] = None,
    json_: Dict[str, str] = None,
    timeout: int = 10,
    *args, **kwargs

) -> requests.Response:
    """
    Make an HTTP request using the requests library.

    Args:
        url (str)
        method (str, optional): The HTTP method to use. Defaults to "GET".
        params (Dict[str, Any], optional): URL parameters to include in the request.
        data (Dict[str, Any], optional): Data to send in the request body.
        headers (Dict[str, str], optional): Headers to include in the request.
        timeout (int, optional): Timeout for the request in seconds. Defaults to 10.

    Returns:
        requests.Response: The response object from the request.

    Raises:
        requests.RequestException
    """
    try:
        response = requests.request(
            method=method.upper(),
            url=url,
            params=params,
            data=data,
            headers=headers,
            timeout=timeout,
            json=json_,
            *args, **kwargs
            
        )
        response.raise_for_status()
        print(f"{json.dumps(response.json(), indent = 2)}")
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        raise
    return response

In [33]:

d

Help on function request in module requests.api:

request(method, url, **kwargs)
    Constructs and sends a :class:`Request <Request>`.
    
    :param method: method for the new :class:`Request` object: ``GET``, ``OPTIONS``, ``HEAD``, ``POST``, ``PUT``, ``PATCH``, or ``DELETE``.
    :param url: URL for the new :class:`Request` object.
    :param params: (optional) Dictionary, list of tuples or bytes to send
        in the query string for the :class:`Request`.
    :param data: (optional) Dictionary, list of tuples, bytes, or file-like
        object to send in the body of the :class:`Request`.
    :param json: (optional) A JSON serializable Python object to send in the body of the :class:`Request`.
    :param headers: (optional) Dictionary of HTTP Headers to send with the :class:`Request`.
    :param cookies: (optional) Dict or CookieJar object to send with the :class:`Request`.
    :param files: (optional) Dictionary of ``'name': file-like-objects`` (or ``{'name': file-tuple}``) for 

In [3]:
# Check a sample URL response
make_request(f"{API_URL}/health")

{
  "status": "OK",
  "deployment_type": "local"
}


## Working with Datasets

In [4]:
# Show datasets to evaluate
make_request(f"{API_URL}/datasets")

{
  "total": 0,
  "items": []
}


In [5]:
# Upload a dataset



In [6]:
# Get information about a dataset

## Working with Experiments

## Creating a Ground Truth Run


In [24]:
my_deployment = make_request(f"{API_URL}/ground-truth/deployments", method="post").json().get("id")


{
  "id": "24d44f4f-3844-4658-99ad-dccdd6ab619e",
  "name": "summarizer",
  "description": "Text summarization model",
  "status": "created",
  "created_at": "2024-07-15T20:26:44.440081Z",
  "updated_at": null
}


In [13]:
# Show deployments

deployments = make_request(f"{API_URL}/ground-truth/deployments/")
deployments

{
  "total": 8,
  "items": [
    {
      "id": "2be368ad-eb6a-4780-bb7f-b4f19c3c6ca1",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-15T17:45:49.056926Z",
      "updated_at": null
    },
    {
      "id": "524507f9-8e34-4a53-8249-bc17573608b2",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-15T17:54:24.963071Z",
      "updated_at": null
    },
    {
      "id": "567459ba-96ce-4ea9-bd0d-1738470776cb",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-15T17:55:42.666413Z",
      "updated_at": null
    },
    {
      "id": "33e90697-7449-416d-a458-22a0f6186d73",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-15T17:56:09.893899Z",
      "updated_at": nul

In [ ]:
my_deployment = deployments

In [8]:
## A way to see all options on the platform 

In [9]:
payload =  """I recently added a beta release, version 7.0b1, of my extension, Clippings for Thunderbird, and selected the option to self-host it so that I can make it available to testers separately while regular users continue to see version 6.3.5, the current release version. However, the beta release is now incorrectly appearing in the Add-ons for Thunderbird public listing. URL to ATN listing: https://addons.thunderbird.net/en-US/thunderbird/addon/clippings-tb/. Thanks for the report, it is best to file an issue with the addons-server: https://github.com/thunderbird/addons-server I have not looked at it in detail, but self-hosted add-ons need an update_url entry in their manifest and that should prevent it from being accepted on ATN. Since we do not sign add-ons, self-hosted add-ons do not need to be submitted to ATN at all.My best-practice advice is:
- remove the beta version from ATN,
- create a dedicated branch/repo which holds the update information (either a branch in each add-on repo you want to self-host, or a single update-repo with the information for all your add-ons)
- host the XPI files either as an "beta" asset in a "github release", or directly as a file in the repo (I think the asset is the better choice),
- make sure that the manifest of your self-hosted XPI points to the correct update.json. John I've removed the beta version of my extension from ATN, and the listing for my extension now shows the current stable release. Thanks for your help!"""

In [42]:
summary = make_request(f"{API_URL}/ground-truth/deployments/{my_deployment}", method="POST", json_={"text": payload}, timeout=30)

{
  "deployment_response": {
    "result": "Clippings for Thunderbird self-hosts a beta release of version 7.0b1. The beta release is now incorrectly appearing in the Add-ons for Thunderbirds public listing. John has removed the beta version from ATN, and the listing for my extension now shows the current stable release."
  }
}


## Model Selection

## Run Experiment

## Show evaluation results